<a href="https://colab.research.google.com/github/vkalvakotamath/gpt_instruct/blob/main/bs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets

In [3]:
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from transformers import PreTrainedTokenizerBase
from typing import Dict, List, Optional

from datasets import load_dataset
import torch

class InstructGPT:
    def __init__(
        self,
        model_name: str = 'gpt2-medium',
        max_length: int = 420
    ):
        self.model_name = model_name
        self.max_length = max_length


        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.model = GPT2LMHeadModel.from_pretrained(model_name)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model.config.pad_token_id = self.tokenizer.pad_token_id



    def prepare_dataset(self):
        # Change this to a better dataset because this one sucks
        dataset = load_dataset("Dahoas/instruct-synthetic-prompt-responses")

        # Combine prompts
        def combine_prompt_response(example):
            full_text = f"{example['prompt']} {self.tokenizer.eos_token} {example['response']}"
            return {'text': full_text}
        dataset = dataset.map(combine_prompt_response, remove_columns=dataset['train'].column_names)
        return dataset

    def tokenize_function(self, examples):
        return self.tokenizer(
            examples['text'],
            truncation=True,

            max_length=self.max_length,
            padding='max_length'
        )

    def prepare_training_data(self, test_size: float = 0.1):

        tokenized_datasets = self.dataset.map(
            self.tokenize_function,
            remove_columns=self.dataset['train'].column_names,

            batched=True,
        )

        # Split into train and validation
        tokenized_datasets = tokenized_datasets['train'].train_test_split(test_size=test_size)

        return tokenized_datasets

    def train(
        self,
        output_dir: str = './instruct-gpt2-model',
        learning_rate: float = 5e-5,
        batch_size: int = 8,
        num_train_epochs: int = 1, # not really sure about this stuff yet, may want to tweak this later
    ):

        weight_decay=0.01

        self.dataset = self.prepare_dataset() # Call prepare_dataset to initialize self.dataset
        tokenized_datasets = self.prepare_training_data()

        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False
        )
        # Training arguments
        training_args = TrainingArguments(
            fp16=True,
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            eval_steps=100,
            prediction_loss_only=True,
            learning_rate=learning_rate,
            weight_decay=weight_decay
        )


        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_datasets['train'],
            eval_dataset=tokenized_datasets['test'],
            data_collator=data_collator,
        )


        trainer.train()
        trainer.save_model()

        # Stuff from Claude because I apparently messed up a bunch of things
    def generate_text(
        self,
        prompt: str,
        max_length: int = 420,
        temperature: float = 0.5,
        top_p: float = 0.9
    ) -> str:
        """
        Generate text using trained model

        Args:
            prompt (str): Input prompt
            max_length (int): Maximum generation length
            temperature (float): Sampling temperature
            top_p (float): Nucleus sampling threshold

        Returns:
            Generated text
        """
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        input_ids = self.tokenizer.encode(prompt, return_tensors='pt').to(device)

        output = self.model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            top_p=top_p,
            no_repeat_ngram_size=2,
            do_sample=True
        )

        return self.tokenizer.decode(output[0], skip_special_tokens=True)



def main():
    instruct_trainer = InstructGPT()
    instruct_trainer.train(
        output_dir='./instruct-gpt2-model',
        learning_rate=5e-5,
        batch_size=8,
        num_train_epochs=1
    )

    prompt = "Give me a few good flirts and flirtatious texts to send to my girlfriend"
    generated_text = instruct_trainer.generate_text(prompt)
    print("Response::", generated_text)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/33203 [00:00<?, ? examples/s]

Map:   0%|          | 0/33203 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: quantum_vaibhav (quantum_vaibhav-bayesian-group). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.802900
1000,1.674100
1500,1.643800
2000,1.617500
2500,1.595900
3000,1.575900
3500,1.558500


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Response:: Give me a few good flirts and flirtatious texts to send to my girlfriend.  

1. "Hey, how are you doing today? Can you tell me about the day's activities? 

 		  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50 51  52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 2

In [4]:
def interactive_chat(model_path='./instruct-gpt2-model'):
    """
    Interactive chat interface for the trained InstructGPT model.

    Args:
        model_path (str): Path to the trained model
    """
    # Initialize model with trained weights
    chatbot = InstructGPT()
    try:
        chatbot.model = GPT2LMHeadModel.from_pretrained(model_path)
        chatbot.model.eval()  # Set to evaluation mode
        print("Model loaded successfully from", model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Falling back to base GPT-2 model")

    print("\nInteractive Chat Session")
    print("Enter 'quit', 'exit', or press Ctrl+C to end the conversation")
    print("=" * 50)

    # Chat loop
    while True:
        try:
            # Get user input
            user_input = input("\nYou: ").strip()

            # Check for exit commands
            if user_input.lower() in ['quit', 'exit']:
                print("\nEnding chat session...")
                break

            if not user_input:
                print("Please enter a prompt!")
                continue

            # Generate response
            print("\nGenerating response...")
            response = chatbot.generate_text(
                prompt=user_input,
                temperature=0.7,  # Slightly higher temperature for more creative responses
                top_p=0.9,
                max_length=150  # Shorter responses for interactive chat
            )

            # Print response
            print("\nBot:", response.strip())

        except KeyboardInterrupt:
            print("\n\nEnding chat session...")
            break
        except Exception as e:
            print(f"\nError occurred: {e}")
            print("Please try again!")

# Usage example
if __name__ == "__main__":
    interactive_chat()

Model loaded successfully from ./instruct-gpt2-model

Interactive Chat Session
Enter 'quit', 'exit', or press Ctrl+C to end the conversation

You: Hi give me a story

Generating response...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Bot: Hi give me a story about how I went to college and how it changed my life

 		  
I went into college as a full-time student in 2000. I was studying for my final exams and had no idea that I would be taking on a major in Psychology. It was an exciting time to be a student and I could not have been more excited. However, things started to go wrong soon after I started classes. My grades started plummeting and my workload increased dramatically. The school administration was extremely difficult to deal with and it became increasingly difficult for me to complete my studies. In the end, I had to take a leave of absence and then a year to recover from my illness. With all of my work and

You: Hi, how are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Generating response...

Bot: Hi, how are you?

 		  
I'm doing great! I'm feeling very good about my new job and excited to get started. I want to thank you for taking the time to read my resume. It's a great way to make sure I am speaking the truth about myself and my qualifications. Good luck!


Sincerely, 

[Your Name] [Hiring Company]
[Company Name Here] [Company Email Address]

1  Contact me at [company name] with any questions or comments.  [Name of Company],  Can you help me find a job for me?  I'd be happy to help.
2  If you're interested in a career change,

You: No fuck


...


Generating response...

Bot: We finish each other's sentences and start to talk.  

I'm going to the beach today. What are your plans? 

 		
My plans are to go to a nearby beach and enjoy the sun. Is there anything I can do to help you plan?
 My plan is to take a few minutes to relax and soak up the ocean breeze. Do you know any good activities I could do?

	  My plan for the day is for me to hike and explore. How about a hike to an isolated spot? I'm sure I have something you could try.
	My next plan would be to explore the local area. There are lots of outdoor activities and activities that you can take part in

You: Explain BRST quantization


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Generating response...

Bot: Explain BRST quantization and how it works

 		  
Quantization is the process of taking the sum of two or more variables and dividing it by the total number of variables. It is used to calculate the average or mean of a range of values, or to divide a series of numbers into smaller and smaller groups. The key concept of BRT is to use the BRTS (Bi-weekly Trained Ratio) to determine the expected return on investments (ROI) of each investment. BRTs can be expressed as an average of the two variables in the series, with the upper and lower bounds being equal. For example, if the variables are 1, 2, 3 and 4, the value 1 + 2 =

You: What is 2+2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Generating response...

Bot: What is 2+2=5?

 		   1  2 + 2 = 5 
	1  3  4 = 6 

	2  5 = 7  (2 + 3)  = 9  6 + 4  + 5
  The answer to this question is: 2 equals 5. Therefore, 2 equal 5 means that 3 is equal to 6. 6 is also equal, so 6 equals 7. Finally, 7 equals 9. Thus, 3 equals 6 and 4 equals 3. This means 3 and 6 are equal.  Therefore 3 = 3, 6 = 4, and 9 = 8. So, the answer is that 2 is 5, but 3 does not equal 6, 4 does,


Ending chat session...


In [1]:
!pip install --upgrade huggingface_hub